In [1]:
from ExtractUrlArticle import ExtractUrlArticle
from Translator import Translator
from newspaper import Article
from googletrans import Translator as googleTranslator
import requests 
import json
import pandas

### Extracteur d'url d'articles 

In [2]:
ExtractorArticle = ExtractUrlArticle()

# Extraction des Url selon les Pays
liste_fr=  pandas.DataFrame(ExtractorArticle.ExtractArticleFr()).url
#liste_de = pandas.DataFrame(ExtractorArticle.ExtractArticleDe()).url
liste_en = pandas.DataFrame(ExtractorArticle.ExtractArticleEn()).url

liste_fa=  pandas.DataFrame(ExtractorArticle.ExtractArticleFa()).url
liste_ar = pandas.DataFrame(ExtractorArticle.ExtractArticleAr()).url

# Création d'un dictionnaire pour stocker les listes
url_dict = {
    'fr': liste_fr,
    #'de': liste_de,
    'en': liste_en,
    'fa': liste_fa,
    'ar': liste_ar
}

# Pour avoir un nombre équitable d'articles côté occident et moyen orient, nous réduisons la liste des média Anglais.
# ==> 63 articles de chaque côté
liste_en = liste_en[:39]

### Récupération des texte et traduction des articles

In [ ]:
#translator = Translator()
tr = googleTranslator()

textes_dict = dict() # Dictionnaire avec comme clé le pays et en valeur la liste des textes d'articles 

for pays in url_dict:
    if pays!="fr":
        textes=list()
        for url in url_dict[pays]:
            article = Article(url)
            try:
                article.download()
                article.parse()
                textes.append(tr.translate(article.text,src=pays,dest='fr').text)
            except Exception as e:
                print(f"Erreur lors du traitement de l'article : {e}")
                continue
        textes_dict[pays] = textes
        
    else:
        textes=list()
        for url in url_dict[pays]:
            article = Article(url)
            try: 
                article.download()
                article.parse()
                textes.append(article.text)
            except Exception as e:
                continue
        textes_dict[pays] = textes
        
        
            

### Création et ecriture de fichiers des articles

In [23]:
# Ajout des articles anglais et français dans la liste d'articles occidentaux

liste_occident = list()
for i in textes_dict['fr']:
    liste_occident.append(i)
for i in textes_dict['en']:
    liste_occident.append(i)
    
# Ajout des articles arabe et iranien  dans la liste d'articles du moyen orient 
liste_orient = list()
for i in textes_dict['ar']:
    liste_orient.append(i)
for i in textes_dict['fa']:
    liste_orient.append(i)

In [11]:
# Création du fichier pour les médias occidentaux 
with open("Articles_Occidentaux.txt", "w",encoding="utf-8") as f:
    for texte in liste_occident:
        f.write(texte+"\n")

# Création du fichier pour les médias du Moyen Orient
with open("Articles_Moyen_Orient.txt", "w",encoding="utf-8") as f:
    for texte in liste_orient:
        f.write(texte+"\n")